# Using SQLAlchemy

In [1]:
import sqlalchemy as db
import pandas as pd
import os
import csv
import seaborn as sns
import matplotlib.pyplot as plt

## Connecting to database

In [2]:
# connecting to database sql_challenge created in postgresql
engine = db.create_engine('postgresql://localhost:5432/housing-price')
connection = engine.connect()
metadata = db.MetaData()

In [3]:
#to find all the tables for the database (it should be empty)
engine.table_names()

/var/folders/ld/_spx9cwj74z4zm921xp1zssh0000gn/T/ipykernel_52417/3558990862.py:2: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


['data_2017', 'prediction', 'data_2019']

## used to_sql to insert data 

In [4]:
# path of cleaned csv files
path_2019=os.path.join('output_census', 'usa_2019_ml.csv')
path_2017=os.path.join('output_census', 'usa_2017_ml.csv')
path_predict=os.path.join('output_census', 'predict.csv')

In [5]:
#For table train


with open(path_2019, 'r') as train:
    data_2019 = pd.read_csv(train)
data_2019.to_sql('data_2019', con=engine, index=False, if_exists='replace')




with open(path_2017, 'r') as test:
    data_2017 = pd.read_csv(test)
data_2017.to_sql('data_2017', con=engine, index=False, if_exists='replace')




with open(path_predict, 'r') as pred:
    prediction = pd.read_csv(pred)
prediction.to_sql('prediction', con=engine, index=False, if_exists='replace')




In [6]:
#getting the tables from database
data_2019 = db.Table('data_2019', metadata, autoload=True, autoload_with=engine)
data_2017 = db.Table('data_2017', metadata, autoload=True, autoload_with=engine)
prediction = db.Table('prediction', metadata, autoload=True, autoload_with=engine)



In [7]:
# columns of census table
data_2019.c.keys()

['Population',
 'Median Age',
 'Household Income',
 'Per Capita Income',
 'Poverty Rate',
 'Unemployment Rate',
 'House Value',
 'House Construction Year',
 'Monthly Owner Cost',
 'Monthly Rent',
 'Public Transport Rate',
 'Personal Transport Rate',
 'High School Rate',
 'College Rate',
 'Uneducated Rate',
 'White Population Rate',
 'Black Population Rate',
 'Hispanic Population Rate',
 'Asian Population Rate',
 'City',
 'Lat',
 'Lng',
 'Housing_units']

In [13]:
#adding primary key
connection.execute('alter table data_2019 add id serial primary key')
connection.execute('alter table data_2017 add id serial primary key')
connection.execute('alter table prediction add id serial primary key')


# Using SQLAlchemy ORM

# Reflect Tables into SQLAlchemy ORM

In [8]:
# Python SQL toolkit and Object Relational Mapper
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect, and_, or_

In [9]:
# create engine to hawaii.sqlite
engine = create_engine('postgresql://localhost:5432/housing-price')
conn=engine.connect()

In [10]:
# Use the Inspector to explore the database and print the table names
inspector=inspect(engine)
inspector.get_table_names()

['data_2019', 'data_2017', 'prediction']

In [14]:
# reflect an existing database into a new model
Base = automap_base()
Base.prepare(engine, reflect=True)

In [15]:
# View all of the classes that automap found
Base.classes.keys()

['prediction', 'data_2019', 'data_2017']

In [16]:
# Save references to each table
data_2019 = Base.classes.data_2019
data_2017 = Base.classes.data_2017
prediction=Base.classes.prediction


In [17]:
# Create our session (link) from Python to the DB
session = Session(engine)

# Exploring election table

In [19]:
#using pd.read_sql to select and display (also able to see columns name)
df_election=pd.read_sql("SELECT * FROM prediction", conn)

df_election.head()

,City,ActualHouseValue,RandomForestPredictedHouseValue,Lat,Lng,id
0,Aguada,87300.0,96204.0,18.36,-67.18,1
1,Aguadilla,122300.0,112118.0,18.45,-67.11,2
2,Maricao,92700.0,86649.0,18.20,-66.90,3
3,Anasco,90300.0,92350.0,18.28,-67.13,4
4,Arecibo,97300.0,103016.0,18.40,-66.68,5


# Close session

In [ ]:
# Close Session
session.close()